<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/RAG_OpenAI/03_vectorstores_and_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

程式是從 [deeplearning.ai](deeplearning.ai) 的 "[LangChain Chat with Your Data](https://learn.deeplearning.ai/courses/langchain-chat-with-your-data/lesson/snupv/introduction)" 取得

"LangChain Chat with Your Data" 課程可在底下連結取得:

  https://learn.deeplearning.ai/courses/langchain-chat-with-your-data/lesson/snupv/introduction

# Vectorstores and Embeddings

Recall the overall workflow for retrieval augmented generation (RAG):

In [46]:
# prompt: !cp -r /content/doc /content/drive/MyDrive/

from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/doc /content/drive/MyDrive/


ls: cannot access 'drive/MyDrive/發佈': No such file or directory


In [2]:
!pip install dotenv

In [3]:
import os
import openai
import sys
from getpass import getpass
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key  = os.environ['OPENAI_API_KEY']

openai.api_key = getpass("請輸入你的 OpenAI API 金鑰：")

請輸入你的 OpenAI API 金鑰：··········


We just discussed `Document Loading` and `Splitting`.

In [8]:
!pip install -U langchain-community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.9 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("./MachineLearning-Lecture01.pdf"),
    PyPDFLoader("./MachineLearning-Lecture01.pdf"),
    PyPDFLoader("./MachineLearning-Lecture02.pdf"),
    PyPDFLoader("./MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [10]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [11]:
splits = text_splitter.split_documents(docs)

In [12]:
len(splits)

208

## Embeddings

Let's take our splits and embed them.

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

<ipython-input-13-c9983b48059e>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [14]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [16]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00


In [17]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [18]:
import numpy as np

In [19]:
np.dot(embedding1, embedding2)

np.float64(0.9631510802407719)

In [20]:
np.dot(embedding1, embedding3)

np.float64(0.7702031204123156)

In [21]:
np.dot(embedding2, embedding3)

np.float64(0.7590539714454778)

## Vectorstores

In [ ]:
# ! pip install chromadb

In [22]:
from langchain.vectorstores import Chroma

In [23]:
persist_directory = 'docs/chroma/'

In [24]:
!rm -rf ./docs/chroma  # remove old database files if any

In [26]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [27]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [28]:
print(vectordb._collection.count())

208


### Similarity Search

In [29]:
question = "is there an email i can ask for help"

In [30]:
docs = vectordb.similarity_search(question,k=3)

In [31]:
len(docs)

3

In [32]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

Let's save this so we can use it later!

In [33]:
vectordb.persist()

<ipython-input-33-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## Failure modes

This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up.

Here are some edge cases that can arise - we'll fix them in the next class.

In [34]:
question = "what did they say about matlab?"

In [35]:
docs = vectordb.similarity_search(question,k=5)

Notice that we're getting duplicate chunks (because of the duplicate `MachineLearning-Lecture01.pdf` in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

`docs[0]` and `docs[1]` are indentical.

In [36]:
docs[0]

Document(metadata={'total_pages': 22, 'page': 8, 'page_label': '9', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': './MachineLearning-Lecture01.pdf'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB licen

In [37]:
docs[1]

Document(metadata={'total_pages': 22, 'moddate': '2008-07-11T11:25:23-07:00', 'page_label': '9', 'page': 8, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationdate': '2008-07-11T11:25:23-07:00', 'title': '', 'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'source': './MachineLearning-Lecture01.pdf'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB licen

We can see a new failure mode.

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [38]:
question = "what did they say about regression in the third lecture?"

In [39]:
docs = vectordb.similarity_search(question,k=5)

In [40]:
for doc in docs:
    print(doc.metadata)

{'total_pages': 16, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationdate': '2008-07-11T11:25:03-07:00', 'page_label': '1', 'moddate': '2008-07-11T11:25:03-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'title': '', 'source': './MachineLearning-Lecture03.pdf', 'author': '', 'page': 0}
{'title': '', 'page': 2, 'source': './MachineLearning-Lecture02.pdf', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:05-07:00', 'author': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'total_pages': 18, 'moddate': '2008-07-11T11:25:05-07:00', 'page_label': '3'}
{'page_label': '18', 'moddate': '2008-07-11T11:25:05-07:00', 'page': 17, 'total_pages': 18, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'title': '', 'source': './MachineLearning-Lecture02.pdf', 'creator': 'PScript5.dll Version 5.2.2', 'author': '', 'creationdate': '2008-07-11T11:25:05-07:00'}
{'title': '', 'author': '', 'page': 8, 'source': './MachineLearning-Lecture01.pdf', 'creator': 'PScript5.dl

In [41]:
print(docs[4].page_content)

into his office and he said, "Oh, professor, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's helped me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "Wow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data networks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutorial in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical thing is the discussion sections. So discussion 
sections will be taught by the TAs, and attendance at discussion sections is o

Approaches discussed in the next lecture can be used to address both!

### 如果是在 Colab 執行這個程式，
### 需要將 docs 子目錄存起來，後續可以直接使用這個 database